# Load dataset and seperate into train/test/validation

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

batch_size = 32
training, validation, test = tfds.load('cats_vs_dogs', split=['train[:70%]', 'train[70%:80%]', 'train[90%:]'], as_supervised=True)

def preprocess(ds):
    ds = ds.cache()
    ds = ds.map(lambda x, y: (tf.image.rgb_to_grayscale(x), y))
    ds = ds.map(lambda x, y: (tf.image.resize(x, (256, 256)), y))
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return ds

training = preprocess(training)
validation = preprocess(validation)
test = preprocess(test)

# Set up model

In [3]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.losses.BinaryCrossentropy(), metrics=['binary_accuracy'])

# Train model

In [4]:
epochs = 5
model.fit(training, epochs=epochs, validation_data=validation)

Epoch 1/5
509/509 [==============================] - 108s 143ms/step - loss: 0.6346 - binary_accuracy: 0.6351 - val_loss: 0.6054 - val_binary_accuracy: 0.6829
Epoch 2/5
509/509 [==============================] - 54s 103ms/step - loss: 0.5078 - binary_accuracy: 0.7513 - val_loss: 0.5054 - val_binary_accuracy: 0.7520
Epoch 3/5
509/509 [==============================] - 55s 104ms/step - loss: 0.4014 - binary_accuracy: 0.8143 - val_loss: 0.5483 - val_binary_accuracy: 0.7323
Epoch 4/5
509/509 [==============================] - 54s 102ms/step - loss: 0.2676 - binary_accuracy: 0.8871 - val_loss: 0.6759 - val_binary_accuracy: 0.7606
Epoch 5/5
509/509 [==============================] - 54s 103ms/step - loss: 0.1309 - binary_accuracy: 0.9496 - val_loss: 1.0869 - val_binary_accuracy: 0.7383


# Evaluate on test set

In [6]:
print(model.metrics_names)
print(model.evaluate(test, verbose=0))

['loss', 'binary_accuracy']
[0.9103648066520691, 0.7691315412521362]
